In [1]:
# 파이토치에서 의 모델 정의

# 계층 : 모듈 또는 모듈을 구성하는 한 개의 계층으로 합성곱층, 선형계층 있다.
# 모듈: 한개 이상의 계층이 모여서 된것, 모듈이 모여서 새로운 모듈을 만들수있따. 
# 모델: 최종적으로 원하는 네트워크로, 한개의 모듈이 모델이 될수도 있다. 


# 신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성되어 있습니다. 
# torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공합니다. 

# PyTorch의 모든 모듈은 nn.Module 의 하위 클래스(subclass) 입니다. 
# 신경망은 다른 모듈(계층; layer)로 구성된 모듈입니다. 
# 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있습니다.

# 이어지는 장에서는 FashionMNIST 데이터셋의 이미지들을 분류하는 신경망을 구성해보겠습니다.

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [5]:
# 클래스 정의하기
# 신경망 모델을 nn.Module 의 하위클래스로 정의하고,
# __init__ 에서 신경망 계층들을 초기화합니다. 
# nn.Module 을 상속받은 모든 클래스는 forward 메소드에
# 입력 데이터에 대한 연산들을 구현합니다.

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
# 모델을 사용하기 위해 입력 데이터를 전달합니다.
# 이는 일부 백그라운드 연산들 과 함께 모델의 forward 를 실행합니다. 
# model.forward() 를 직접 호출하지 마세요!

# 모델에 입력을 전달하여 호출하면 2차원 텐서를 반환합니다. 
# 2차원 텐서의 dim=0은 각 분류(class)에 대한 원시(raw) 예측값 10개가,
# dim=1에는 각 출력의 개별 값들이 해당합니다.
# 원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻습니다.


X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[-0.0166,  0.0314, -0.0605, -0.0164,  0.0262,  0.0417, -0.0387, -0.0495,
         -0.0201,  0.0150]], grad_fn=<AddmmBackward0>)
tensor([[0.0992, 0.1040, 0.0949, 0.0992, 0.1035, 0.1051, 0.0970, 0.0959, 0.0988,
         0.1023]], grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([5])


In [8]:
# 모델 계층(Layer)
# FashionMNIST 모델의 계층들을 살펴보겠습니다. 
# 이를 설명하기 위해, 28x28 크기의 이미지 3개로 구성된 미니배치를 
# 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 알아보겠습니다.

input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [9]:
# nn.Flatten
# nn.Flatten 계층을 초기화하여 
# 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환합니다. 
# (dim=0의 미니배치 차원은 유지됩니다.)

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [10]:
# nn.Linear
# 선형 계층 은 저장된 가중치(weight)와 편향(bias)을 사용하여 
# 입력에 선형 변환(linear transformation)을 적용하는 모듈입니다.


layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [11]:
# nn.ReLU
# 비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한 관계(mapping)
# 를 만듭니다. 비선형 활성화는 선형 변환 후에 적용되어 
# 비선형성(nonlinearity) 을 도입하고, 
# 신경망이 다양한 현상을 학습할 수 있도록 돕습니다.


# 이 모델에서는 nn.ReLU 를 선형 계층들 사이에 사용하지만, 
# 모델을 만들 때는 비선형성을 가진 다른 활성화를 도입할 수도 있습니다.

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")


Before ReLU: tensor([[ 0.1371, -0.1086, -0.0744,  0.4058, -0.0051, -0.1039, -0.3381,  0.0286,
         -0.0059, -0.0684,  0.3118,  0.3865,  0.8160,  0.0628,  0.1784,  0.4473,
          0.0912, -0.2312,  0.6481, -0.4300],
        [-0.0612,  0.1155, -0.0320,  0.4041, -0.0189, -0.4277, -0.0489, -0.4132,
         -0.4923,  0.0979,  0.4057,  0.4449,  0.8338, -0.0227,  0.0515,  0.4630,
          0.0968, -0.2208,  0.5646,  0.3240],
        [-0.0251,  0.1190, -0.1727,  0.4327,  0.0871, -0.3950, -0.3111, -0.1445,
          0.3360,  0.0712,  0.1996,  0.6683,  0.9212, -0.2042,  0.0240,  0.4750,
          0.0121, -0.3972,  0.7208, -0.1379]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1371, 0.0000, 0.0000, 0.4058, 0.0000, 0.0000, 0.0000, 0.0286, 0.0000,
         0.0000, 0.3118, 0.3865, 0.8160, 0.0628, 0.1784, 0.4473, 0.0912, 0.0000,
         0.6481, 0.0000],
        [0.0000, 0.1155, 0.0000, 0.4041, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0979, 0.4057, 0.4449, 0.8338, 0.0000, 0.05

In [12]:
# nn.Sequential 은 순서를 갖는 모듈의 컨테이너입니다. 
# 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달됩니다. 
# 순차 컨테이너(sequential container)를 사용하여 
# 아래의 seq_modules 와 같은 신경망을 빠르게 만들 수 있습니다.

seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [13]:
# nn.Softmax
# 신경망의 마지막 선형 계층은 nn.Softmax 모듈에 
# 전달될 ([-infty, infty] 범위의 원시 값(raw value)인) logits 를 반환합니다. 
# logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록
# [0, 1] 범위로 비례하여 조정(scale)됩니다. 
# dim 매개변수는 값의 합이 1이 되는 차원을 나타냅니다.

softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [14]:
# 모델 매개변수
# 신경망 내부의 많은 계층들은 매개변수화(parameterize) 됩니다.
# 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어집니다. 
# nn.Module 을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며,
# 모델의 parameters() 및 named_parameters() 메소드로 
# 모든 매개변수에 접근할 수 있게 됩니다.

# 이 예제에서는 각 매개변수들을 순회하며(iterate), 
# 매개변수의 크기와 값을 출력합니다.

print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0299, -0.0163, -0.0299,  ...,  0.0268,  0.0040,  0.0062],
        [-0.0270,  0.0079, -0.0013,  ...,  0.0313, -0.0032,  0.0337]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0048, -0.0006], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0381,  0.0247, -0.0141,  ...,  0.0380,  0.0423,  0.0272],
        [ 0.0055, -0.0289,  0.0165,  ..., -0.0093, -0.0159, -0.0095]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 